In [1]:
import json
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.metrics import euclidean_distances
from rouge import Rouge
import sklearn
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
from nltk import word_tokenize
from difflib import SequenceMatcher
import enchant
from nltk.corpus import stopwords

In [2]:
def cosine_similarity(X,Y):
    
    X_list = word_tokenize(X) 
    Y_list = word_tokenize(Y)
    sw = stopwords.words('english') 
    l1 =[];l2 =[]
    X_set = {w for w in X_list if not w in sw} 
    Y_set = {w for w in Y_list if not w in sw}


    rvector = X_set.union(Y_set) 
    for w in rvector:
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0)
        if w in Y_set: l2.append(1)
        else: l2.append(0)
    c = 0


    for i in range(len(rvector)):
            c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    return cosine

In [3]:
def Rouge_Score(hypothesis, reference):
    
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    
    return scores

In [4]:
def Jaccard_Similarity(doc1, doc2): 
    

    words_doc1 = set(doc1.lower().split()) 
    words_doc2 = set(doc2.lower().split())
    intersection = words_doc1.intersection(words_doc2)
    union = words_doc1.union(words_doc2)
    
    return float(len(intersection)) / len(union)

In [5]:
def sequence_matcher(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [6]:
def res(a,b):
    return len(set(a.split(' ')) & set(b.split(' '))) / float(len(set(a.split(' ')) | set(b.split(' ')))) *100

In [7]:
a = 'hello how are you'
b = 'i am very good how you'
set(a.split(' ')) & set(b.split(' '))

{'how', 'you'}

In [52]:
# float(len(set(a.split(' ')) | set(b.split(' '))))
set(a.split(' ')) | set(b.split(' '))

{'am', 'are', 'good', 'hello', 'how', 'i', 'very', 'you'}

In [34]:
def pos_tagging(a):
    
    text = word_tokenize(a.lower())
    t = nltk.pos_tag(text)
    d = enchant.Dict("en_US")
    temp = []
    s = ' '
    
    for i in t:
        if i[1] == 'NNS':
#         or i[1] == 'NNS' or i[1] == 'JJ':
            if i[0] not in stopwords.words('english'):
                temp.append(i[0])
    
    return s.join(set(temp))

In [9]:
with open('Task C Train Data.json') as f:
    data = json.load(f)

In [25]:
print()

dialogues meetings annotators transcripts participants


In [36]:
a = "finished evaluation framework. improved domain adaptation and is working on next version of ASR. systems are better with mp3 compression instead of flac format"
pos_tagging(a)

'systems'

In [14]:
data_list = []

for a,b in data.items():
    
    data_list.append([
        a,  
        pos_tagging(b['min_0']), 
        pos_tagging(b['min_1']),
        b['labels']
    ])

In [15]:
df = pd.DataFrame(data_list, columns= ['id', 'min_0', 'min_1', 'label'])

In [31]:
# df.head()

In [32]:
temp = data_list
for j in temp:
    j.append(cosine_similarity(j[1], j[2]))
    j.append(Jaccard_Similarity(j[1], j[2]))
    j.append(Rouge_Score(j[1], j[2])[0]['rouge-1']['f'])
    j.append(Rouge_Score(j[1], j[2])[0]['rouge-l']['f'])
    j.append(sequence_matcher(j[1], j[2]))
    j.append(res(j[1], j[2]))


In [53]:
# temp[0]

In [40]:
df1 = pd.DataFrame(temp, columns= ['id', 'min_0', 'min_1', 'label', 'cosine similarity', 
                                   'jaccard similarity', 'rouge 1','rouge l', 
                                   'sequence matcher', 'res'])

In [43]:
df1.shape

(933, 10)

In [46]:
k = []
for i in range(df1.shape[0]):
    if df1['label'][i] not in [True, False]:
        k.append(i)

In [47]:
df2 = df1.drop(df1.index[k])

In [54]:
df2.head()

,id,min_0,min_1,label,cosine similarity,jaccard similarity,rouge 1,rouge l,sequence matcher,res
0,doc_0,segmenter aggtable colleague instances compute...,consents language transcripts englishcentric c...,False,0.000000,0.000000,0.000000,0.000000,0.020619,0.000000
1,doc_1,proposal annotated submission meetings dialogu...,segmenter sentence conference useissue sentenc...,False,0.034300,0.015152,0.029851,0.029851,0.010870,1.515152
2,doc_2,protocol afternoon language december meetings ...,dissemination multilingual complete components...,False,0.075593,0.025641,0.050000,0.050000,0.161369,2.564103
3,doc_3,startingrunningslackcom romanian speakers gene...,videoscreen colleagues language vigilant multi...,True,0.200821,0.107692,0.194444,0.194444,0.007884,10.769231
4,doc_4,measures important simplify instruction extrac...,workflows meetings language evolution integrat...,False,0.104257,0.049180,0.093750,0.062500,0.005908,4.918033


In [55]:
final_data = df2.drop(['id', 'min_0','min_1'], axis = 1)
# test_scores = df1.drop(['min_0','min_1'], axis = 1)

In [ ]:
test_scores

In [56]:
final_data.head()

,label,cosine similarity,jaccard similarity,rouge 1,rouge l,sequence matcher,res
0,False,0.000000,0.000000,0.000000,0.000000,0.020619,0.000000
1,False,0.034300,0.015152,0.029851,0.029851,0.010870,1.515152
2,False,0.075593,0.025641,0.050000,0.050000,0.161369,2.564103
3,True,0.200821,0.107692,0.194444,0.194444,0.007884,10.769231
4,False,0.104257,0.049180,0.093750,0.062500,0.005908,4.918033


In [320]:
# test_scores.to_csv('test_scores.csv')

In [66]:
final_data.to_csv('Task c train data scores.csv')

In [58]:
true = final_data[final_data['label'] == True]

In [59]:
false = final_data[final_data['label'] == False]

In [60]:
true_scores = []
for i in true:
    true_scores.append([true[i]])

In [65]:
false.shape

(660, 7)

In [161]:
cst = np.array(true_scores[1])

In [162]:
np.mean(cst)

0.1101300353005222

In [160]:
false_scores = []
for i in false:
    false_scores.append([false[i]])

In [163]:
csf = np.array(false_scores[1])

In [164]:
np.mean(csf)

0.040827184963721036